# Responsible AI Model Training
責任ある AI を考慮したモデル構築を行います。データは [Adult Census](https://archive.ics.uci.edu/ml/datasets/adult) をローンの負債・不履行の履歴データとして擬似的に扱います。

## アジェンダ
1. データ準備
    - 利用するサンプルデータのロード、データ前処理のパイプライン作成を行います。
2. モデル構築
    - 説明性、解釈可能性、公平性を考慮したモデルを構築します。

## 1. データ準備
shap ライブラリから提供されているデータ前処理済みのデータを利用します。

In [ ]:
import shap
from helper import ebm_preserve_global, ebm_preserve_local, ebm_preserve_perf

# Load the adult cencus dataset
X_raw, Y = shap.datasets.adult()
print ("X_raw shape:", X_raw.shape)
X_raw.head()


In [ ]:
# 欠損値の有無
X_raw.isna().sum().sum()

In [ ]:
import numpy as np
print(X_raw.dtypes)
categorical_features_indices = np.where(np.logical_or(X_raw.dtypes == np.int8, X_raw.dtypes == np.int32, X_raw.dtypes == np.int64))[0]

print('カテゴリ変数のインデックス:',categorical_features_indices)

numeric_features_indices = np.where(X_raw.dtypes == np.float32)[0]
numeric_features_indices
print('数値変数のインデックス:',numeric_features_indices)

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# データ加工パイプライン
column_transformer = ColumnTransformer ([
    ('onehot', OneHotEncoder(handle_unknown='ignore'),
    categorical_features_indices),
    ('scaler', StandardScaler(),
    numeric_features_indices)
])

In [ ]:
feature_names = X_raw.columns.to_list()
feature_types = ['categorical' if i in categorical_features_indices else 'continuous' for i in range(len(feature_names))]
print(list(zip(feature_names, feature_types)))

In [ ]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
print("ラベルエンコーディング前:",Y) # --> [False False False  ... False False True]
Y=le.fit_transform(Y)
print("ラベルエンコーディング後:",Y) # --> [0 0 0  ... 0 0 1]

In [ ]:
from sklearn.model_selection import train_test_split

A=X_raw[['Sex']] # Sensitive な変数

X_train, X_test, Y_train, Y_test, A_train, A_test = train_test_split(
    X_raw, Y, A,
    test_size=0.2, random_state=0, stratify=Y)

X_train.reset_index(drop=True)
X_test.reset_index(drop=True)
A_train.reset_index(drop=True)
A_test.reset_index(drop=True)

print("X_raw shape: {}, X_train shape: {}, X_test shape: {}".format(
    X_raw.shape, X_train.shape, X_test.shape))
    
# test dataframe: features enrichment
import pandas as pd

pandas_warnings=pd.get_option('mode.chained_assignment')
# to avoid warning 'A value is trying to be set on a copy of a slice from a DataFrame'

pd.set_option('mode.chained_assignment', None)

# improve labels by replacing numbers with labels
A_test.Sex.loc[(A_test['Sex']==0)] = 'female'
A_test.Sex.loc[(A_test['Sex']==1)] = 'male'



pd.set_option('mode.chained_assignment', pandas_warnings)

A_test.head()

## 2. モデル構築

### 1. 通常の勾配ブースティングモデルの構築
CatBoost によるモデル学習を行います。

In [ ]:
# Train your third classification model with Catboost Classifier
from catboost import CatBoostClassifier # !pip install catboost==0.18.1

model_1 = CatBoostClassifier(
    random_seed=42, logging_level="Silent", iterations=150)


pipeline_1 = Pipeline(steps=[
    ('preprocessor', column_transformer),
    ('classifier_CBC', model_1)])

catboost_predictor = pipeline_1.fit(X_train, Y_train)

print('catboost_predictor.score:', catboost_predictor.score(X_test, Y_test))

### 2. 解釈可能性の高いモデルの構築 (Glass-box approach)
[interpret]() ライブラリに含まれる一般化加法モデルの推定アルゴリズム Explainable Boosting Machine (aka EBM) を用いたモデル開発を行います。

In [ ]:
from interpret import show
from interpret.perf import ROC
from interpret.glassbox import ExplainableBoostingClassifier
seed = 1234

#  No pipeline needed due to EBM handling string datatypes
ebm_predictor = ExplainableBoostingClassifier(feature_types = feature_types, random_state=seed, interactions=4)
ebm_predictor.fit(X_train, Y_train)

In [ ]:
ebm_global = ebm_predictor.explain_global(name='EBM')

ebm_preserve_global(ebm_global, 'ebm_global') # for vscode
show(ebm_global)

In [ ]:
ebm_local = ebm_predictor.explain_local(X_test[:10], Y_test[:10], name='EBM')

ebm_preserve_local(ebm_local, 'ebm_local') # for vscode
show(ebm_local)

In [ ]:
ebm_perf = ROC(ebm_predictor.predict_proba).explain_perf(X_test, Y_test, name='EBM')
ebm_preserve_perf(ebm_perf, 'ebm_perf') # for vscode
show(ebm_perf)

### 3. 構築済みモデルに対する説明性の付与 (Black-box approach)
構築済みモデルをブラックボックスとして扱いつつ、説明性を付与していきます。

#### 3.1 説明性の付与と誤差分析
[Interpret-community](https://github.com/interpretml/interpret-community) ライブラリを用いて、Catboost で構築された勾配ブースティングモデルへの説明性の付与と誤差分析を行います。

In [ ]:
from raiwidgets import ExplanationDashboard
from interpret.ext.blackbox import TabularExplainer

# explain predictions on your local machine
# "features" and "classes" fields are optional
explainer = TabularExplainer(catboost_predictor, 
                             X_train)

# explain overall model predictions (global explanation)
global_explanation = explainer.explain_global(X_test)

# ExplanationDashboard(global_explanation, catboost_predictor)
ExplanationDashboard(global_explanation, catboost_predictor, dataset=X_test, true_y=Y_test)

次にモデルの誤差を分析して、潜在的なリスクが何か確認を行います。精度が悪いコホートについては、データ量や質の改善を行い、潜在的なリスクを軽減していきます。

In [ ]:
from raiwidgets import ErrorAnalysisDashboard
ErrorAnalysisDashboard(global_explanation, catboost_predictor, dataset=X_test, true_y=Y_test)

また、これらの一連の流れは統合されたダッシュボードで表現することもできます。

In [ ]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights

train_data = X_train.copy()
train_data["income"] = Y_train

test_data =  X_test.copy()
test_data["income"] = Y_test

categorical_features = X_train.columns[categorical_features_indices].to_list()
target_feature = "income"


# データや目的変数などの情報
rai_insights = RAIInsights(pipeline_1, train_data, test_data, target_feature, 'classification',
                               categorical_features=categorical_features, maximum_rows_for_test=7000)

In [ ]:
# モデル説明性 (InterpretML)
rai_insights.explainer.add()
# モデル誤差解析 (Error Analysis)
rai_insights.error_analysis.add()

In [ ]:
rai_insights.compute()

In [ ]:
ResponsibleAIDashboard(rai_insights)

### 3.2 公平性の評価と軽減
Fairlearn を用いてモデルの公平性の評価を行い、必要に応じて不公平性を軽減するモデルを構築します。

In [ ]:
# 公平性の評価
from raiwidgets import FairnessDashboard
Y_pred = catboost_predictor.predict(X_test)
FairnessDashboard(sensitive_features=A_test,
                  y_true=Y_test,
                  y_pred=Y_pred)

不公平性を軽減する方法として `削除` と `後処理` の 2 種類があります。ここでは `削除` を用いて想いづけされたデータを用いたモデルの再学習を行います。その後、精度と不均衡のトレードオフを確認し、適したモデルを選択します。

In [ ]:
# 不公平性の軽減
from fairlearn.reductions import GridSearch
from fairlearn.reductions import DemographicParity, ErrorRate

sweep = GridSearch(
    model_1,
    constraints=DemographicParity(),
    grid_size=70)

sweep.fit(X_train, Y_train, sensitive_features=A_train.Sex)

In [ ]:
from raiwidgets import FairnessDashboard
mitigated_predictors = sweep.predictors_

ys_mitigated_predictors = {} # it contains (<model_id>, <predictions>) pairs

# the original prediction:
ys_mitigated_predictors["census_unmitigated"]=catboost_predictor.predict(X_test)

base_predictor_name="mitigated_predictor_{0}"
model_id=1

for mp in mitigated_predictors:
    id=base_predictor_name.format(model_id)
    ys_mitigated_predictors[id]=mp.predict(X_test)
    model_id=model_id+1
    
FairnessDashboard(
    sensitive_features=A_test,
    y_true=Y_test,
    y_pred=ys_mitigated_predictors)